In [1]:
!pip install textstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.7 MB/s eta 0:00:00


In [2]:
# Install required Python libraries using pip.
# - gradio: to create web-based user interfaces for machine learning apps
# - textstat: to calculate readability scores and text statistics
# - exifread: to extract metadata (EXIF data) from images
# - plotly: to create interactive visualizations
!pip install gradio textstat exifread plotly

# Import the Natural Language Toolkit (NLTK), a library for text processing
import nltk

# Download the 'punkt' tokenizer model for sentence and word tokenization
nltk.download('punkt')

# Download the list of common stopwords (e.g., "the", "is", "and") in English
nltk.download('stopwords')

# Download the WordNet lexical database for lemmatization and synonym lookup
nltk.download('wordnet')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
# Standard and OS-related
import os  # Provides a way of using operating system dependent functionality like reading file paths
import logging  # Used to log messages for debugging or tracking
import math  # Provides mathematical functions like floor, ceil, etc.

# Computer Vision and Image Handling
import cv2  # OpenCV library for image and video processing
import numpy as np  # Fundamental package for numerical computing and arrays
from PIL import Image  # Python Imaging Library for image manipulation

# PyTorch and torchvision for deep learning and image models
import torch  # Core PyTorch library
import torchvision.models as models  # Pretrained models like ResNet, VGG, etc.
import torchvision.transforms as transforms  # Common image transformations (resize, normalize, etc.)

# Scipy for scientific computing and statistics
from scipy import stats  # For statistical functions like mode, z-score, etc.

# JSON handling
import json  # Read and write JSON data

# Typing for better type hinting in functions
from typing import Dict, List, Tuple, Union, Any

# Suppress warnings if needed
import warnings  # Issue warning messages (can be used to suppress or customize warnings)

# Text and String processing
import re  # Regular expressions for pattern matching in strings
import unicodedata  # Handle Unicode characters, useful in text normalization
import string  # String utilities like punctuation, ascii_letters, etc.

# NLTK – Natural Language Toolkit
import nltk  # Base NLTK package for text processing
from nltk.corpus import stopwords, wordnet  # Stopwords for filtering; WordNet for lexical database
from nltk.tokenize import word_tokenize, sent_tokenize  # Sentence and word tokenizers

# Counter from collections for word frequency counting
from collections import Counter  # Count occurrences of elements in an iterable (like words)

# Plotly for interactive plotting
import plotly.graph_objects as go  # For low-level chart building
import plotly.express as px  # High-level interface for quick plotting

# Textstat for readability scores
from textstat import flesch_reading_ease  # Readability score metric (Flesch Reading Ease)


# Gradio specific import
import gradio as gr  # Gradio is a Python library used to build simple and interactive web interfaces
                     # for machine learning models, data apps, or any Python function.

# Suppress all warnings (optional)
import warnings
warnings.filterwarnings('ignore')  # This line disables all warning messages that Python or libraries may show.
                                   # It helps make the output cleaner, but use it cautiously during debugging.

# ------------------------------------------------------------------
# Custom JSON encoder class for handling NumPy-specific data types
# JSON (JavaScript Object Notation) is a common format for saving or transmitting data.
# However, NumPy data types (like np.int32, np.float64, np.ndarray) are not JSON serializable by default.
# This custom encoder converts them into native Python types so they can be properly saved or shared.
# ------------------------------------------------------------------

class NumpyEncoder(json.JSONEncoder):  # Inheriting from the built-in JSONEncoder class
    def default(self, obj):
        # If the object is a NumPy integer (e.g., np.int32, np.int64), convert it to a regular Python int
        if isinstance(obj, np.integer):
            return int(obj)

        # If the object is a NumPy float (e.g., np.float32, np.float64), convert it to a regular Python float
        elif isinstance(obj, np.floating):
            return float(obj)

        # If the object is a NumPy array (e.g., np.array([...]), convert it to a regular Python list
        elif isinstance(obj, np.ndarray):
            return obj.tolist()

        # If the object is a NumPy boolean (e.g., np.bool_), convert it to a regular Python bool
        elif isinstance(obj, bool):
            return bool(obj)

        # For all other data types, use the default encoding behavior of the parent class
        return super(NumpyEncoder, self).default(obj)

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ==================== IMAGE DETECTION ====================

class ImageDetector:
    def __init__(self):
        # Initialize the model for feature extraction
        try:
            self.model = models.resnet50(pretrained=True)
            self.model = torch.nn.Sequential(*list(self.model.children())[:-1])  # Remove last layer
            self.model.eval()
        except Exception as e:
            logger.warning(f"Could not load ResNet model: {e}. Using fallback methods.")
            self.model = None

        # Define image transformations
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

        # Initialize feature weights
        self.feature_weights = {
            'noise_analysis': 0.3,
            'metadata_analysis': 0.2,
            'deep_features': 0.3,
            'statistical_analysis': 0.2
        }

    def analyze_image(self, image_path: str) -> Dict:
        """Analyze an image to determine if it's AI-generated."""
        try:
            # Load and preprocess image
            image = Image.open(image_path).convert('RGB')
            img_array = np.array(image)

            # Perform various analyses
            noise_score = self._analyze_noise_patterns(img_array)
            metadata_score = self._analyze_metadata(image_path)
            deep_features = self._extract_deep_features(image)
            statistical_score = self._analyze_statistical_features(img_array)

            # Calculate AI indicators
            ai_indicators = {
                'high_noise_consistency': bool(noise_score > 0.7),
                'missing_metadata': bool(metadata_score > 0.6),
                'deep_feature_match': bool(deep_features > 0.65),
                'statistical_anomaly': bool(statistical_score > 0.6)
            }

            # Calculate final score
            final_score = float(
                self.feature_weights['noise_analysis'] * noise_score +
                self.feature_weights['metadata_analysis'] * metadata_score +
                self.feature_weights['deep_features'] * deep_features +
                self.feature_weights['statistical_analysis'] * statistical_score
            )

            # Add AI indicator bonus
            ai_indicator_bonus = sum(ai_indicators.values()) * 0.1
            final_score = min(final_score + ai_indicator_bonus, 1.0)

            return {
                'is_ai_generated': bool(final_score > 0.55),
                'confidence_score': float(final_score),
                'ai_indicators': ai_indicators,
                'detailed_scores': {
                    'noise_analysis': float(noise_score),
                    'metadata_analysis': float(metadata_score),
                    'deep_features': float(deep_features),
                    'statistical_analysis': float(statistical_score)
                },
                'visualizations': self._generate_image_visualizations(
                    noise_score, metadata_score, deep_features, statistical_score, final_score
                )
            }

        except Exception as e:
            logger.error(f"Error analyzing image: {str(e)}")
            return {
                'is_ai_generated': False,
                'confidence_score': 0.0,
                'error': str(e)
            }

    def _analyze_noise_patterns(self, img_array: np.ndarray) -> float:
        """Analyze noise patterns in the image."""
        try:
            # Convert to grayscale for analysis
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)

            # Calculate noise metrics
            # 1. High frequency noise analysis
            laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()

            # 2. Texture analysis using local binary patterns
            def local_binary_pattern(image, radius=1, n_points=8):
                h, w = image.shape
                lbp = np.zeros((h, w), dtype=np.uint8)
                for i in range(radius, h - radius):
                    for j in range(radius, w - radius):
                        center = image[i, j]
                        code = 0
                        for k in range(n_points):
                            angle = 2 * np.pi * k / n_points
                            x = int(i + radius * np.cos(angle))
                            y = int(j + radius * np.sin(angle))
                            if 0 <= x < h and 0 <= y < w:
                                if image[x, y] >= center:
                                    code |= (1 << k)
                        lbp[i, j] = code
                return lbp

            lbp = local_binary_pattern(gray)
            lbp_hist = np.histogram(lbp.ravel(), bins=256)[0]
            lbp_uniformity = np.sum(lbp_hist ** 2) / (lbp_hist.sum() ** 2)

            # 3. Frequency domain analysis
            f_transform = np.fft.fft2(gray)
            f_shift = np.fft.fftshift(f_transform)
            magnitude_spectrum = np.abs(f_shift)

            # Calculate energy distribution
            center = np.array(magnitude_spectrum.shape) // 2
            y, x = np.ogrid[:magnitude_spectrum.shape[0], :magnitude_spectrum.shape[1]]
            mask = (x - center[1])**2 + (y - center[0])**2 <= (min(center) * 0.3)**2

            low_freq_energy = np.sum(magnitude_spectrum[mask])
            total_energy = np.sum(magnitude_spectrum)
            freq_ratio = low_freq_energy / total_energy if total_energy > 0 else 0

            # Combine metrics
            noise_score = 0.0

            # High Laplacian variance suggests less AI processing
            if laplacian_var > 1000:
                noise_score += 0.3
            elif laplacian_var < 100:
                noise_score += 0.7  # Very smooth, likely AI

            # High LBP uniformity suggests AI generation
            if lbp_uniformity > 0.8:
                noise_score += 0.4

            # Frequency analysis
            if freq_ratio > 0.8:  # Too much low frequency content
                noise_score += 0.3

            return min(noise_score, 1.0)

        except Exception as e:
            logger.warning(f"Error in noise analysis: {e}")
            return 0.5  # Default score if analysis fails

    def _analyze_metadata(self, image_path: str) -> float:
        """Analyze image metadata for AI generation indicators."""
        try:
            # Try to read EXIF data
            try:
                import exifread
                with open(image_path, 'rb') as f:
                    tags = exifread.process_file(f)
            except ImportError:
                logger.warning("exifread not available, using PIL for metadata")
                image = Image.open(image_path)
                tags = image._getexif() or {}

            metadata_score = 0.0

            # Check for missing common metadata
            common_tags = ['DateTime', 'Camera', 'Software', 'GPS']
            missing_tags = 0

            if isinstance(tags, dict):
                tag_keys = [str(key).lower() for key in tags.keys()]
                for tag in common_tags:
                    if not any(tag.lower() in key for key in tag_keys):
                        missing_tags += 1
            else:
                missing_tags = len(common_tags)  # Assume all missing if no tags

            # More missing metadata = higher AI score
            metadata_score += (missing_tags / len(common_tags)) * 0.6

            # Check for AI-specific software signatures
            software_indicators = ['adobe', 'midjourney', 'dall-e', 'stable diffusion', 'gpt']
            if isinstance(tags, dict):
                for key, value in tags.items():
                    value_str = str(value).lower()
                    for indicator in software_indicators:
                        if indicator in value_str:
                            metadata_score += 0.4
                            break

            # Check file properties
            image = Image.open(image_path)

            # Common AI generation resolutions
            ai_resolutions = [(512, 512), (1024, 1024), (768, 768), (1024, 768)]
            if image.size in ai_resolutions:
                metadata_score += 0.2

            # Check for perfect aspect ratios (common in AI)
            width, height = image.size
            aspect_ratio = width / height
            if abs(aspect_ratio - 1.0) < 0.01 or abs(aspect_ratio - 1.5) < 0.01:
                metadata_score += 0.1

            return min(metadata_score, 1.0)

        except Exception as e:
            logger.warning(f"Error in metadata analysis: {e}")
            return 0.5

    def _extract_deep_features(self, image: Image.Image) -> float:
        """Extract deep features using ResNet model."""
        try:
            if self.model is None:
                return 0.5  # Default if model not available

            # Preprocess image
            input_tensor = self.transform(image).unsqueeze(0)

            # Extract features
            with torch.no_grad():
                features = self.model(input_tensor)
                features = features.squeeze().numpy()

            # Analyze feature characteristics
            feature_score = 0.0

            # Calculate feature statistics
            mean_activation = np.mean(features)
            std_activation = np.std(features)
            sparsity = np.sum(features == 0) / len(features)

            # AI-generated images often have different activation patterns
            # These thresholds are heuristic and would need validation
            if mean_activation > 0.5:  # High average activation
                feature_score += 0.3

            if std_activation < 0.2:  # Low variation in activations
                feature_score += 0.3

            if sparsity > 0.7:  # High sparsity
                feature_score += 0.2

            # Analyze feature distribution
            hist, _ = np.histogram(features, bins=50)
            hist_normalized = hist / np.sum(hist)
            entropy = -np.sum(hist_normalized * np.log(hist_normalized + 1e-10))

            # Lower entropy might indicate AI generation
            if entropy < 3.0:
                feature_score += 0.2

            return min(feature_score, 1.0)

        except Exception as e:
            logger.warning(f"Error in deep feature extraction: {e}")
            return 0.5

    def _analyze_statistical_features(self, img_array: np.ndarray) -> float:
        """Analyze statistical features of the image."""
        try:
            statistical_score = 0.0

            # Color distribution analysis
            for channel in range(3):  # RGB channels
                channel_data = img_array[:, :, channel].flatten()

                # Calculate statistics
                mean_val = np.mean(channel_data)
                std_val = np.std(channel_data)
                skewness = stats.skew(channel_data)
                kurtosis = stats.kurtosis(channel_data)

                # AI images often have specific statistical properties
                # Normal distribution characteristics
                if abs(skewness) < 0.1:  # Very symmetric
                    statistical_score += 0.1

                if abs(kurtosis) < 0.1:  # Normal kurtosis
                    statistical_score += 0.1

                # Unusual standard deviation
                if std_val < 20 or std_val > 80:
                    statistical_score += 0.1

            # Edge analysis
            gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            edges = cv2.Canny(gray, 100, 200)
            edge_density = np.sum(edges > 0) / edges.size

            # AI images often have specific edge characteristics
            if edge_density < 0.05 or edge_density > 0.3:
                statistical_score += 0.2

            # Gradient analysis
            grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
            gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

            grad_mean = np.mean(gradient_magnitude)
            grad_std = np.std(gradient_magnitude)

            # Unusual gradient patterns
            if grad_std / (grad_mean + 1e-10) > 2.0:
                statistical_score += 0.2

            return min(statistical_score, 1.0)

        except Exception as e:
            logger.warning(f"Error in statistical analysis: {e}")
            return 0.5

    def _generate_image_visualizations(self, noise_score, metadata_score, deep_features, statistical_score, final_score):
        """Generate visualizations for image analysis results."""
        visualizations = {}

        try:
            # Scores bar chart
            scores = {
                'Noise Analysis': noise_score,
                'Metadata Analysis': metadata_score,
                'Deep Features': deep_features,
                'Statistical Analysis': statistical_score
            }

            fig_scores = px.bar(
                x=list(scores.keys()),
                y=list(scores.values()),
                labels={'x': 'Analysis Type', 'y': 'Score'},
                title='Image Analysis Scores',
                color=list(scores.values()),
                color_continuous_scale='RdYlBu_r'
            )
            fig_scores.update_layout(showlegend=False)
            visualizations['analysis_scores'] = fig_scores

            # Confidence gauge
            fig_gauge = go.Figure(go.Indicator(
                mode="gauge+number",
                value=final_score,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "AI Generation Confidence"},
                gauge={'axis': {'range': [0, 1]},
                       'steps': [
                           {'range': [0, 0.3], 'color': "lightgreen"},
                           {'range': [0.3, 0.7], 'color': "yellow"},
                           {'range': [0.7, 1], 'color': "lightcoral"}],
                       'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 0.55}}
            ))
            visualizations['confidence_gauge'] = fig_gauge

        except Exception as e:
            logger.warning(f"Error generating visualizations: {e}")

        return visualizations

# ==================== TEXT DETECTION ====================

class TextPreprocessor:
    def __init__(self):
        self.contractions = {
            "n't": " not", "'ll": " will", "'re": " are", "'ve": " have",
            "'m": " am", "'d": " would", "'s": " is", "won't": "will not",
            "can't": "cannot", "shouldn't": "should not", "wouldn't": "would not"
        }

    def preprocess(self, text: str) -> str:
        """Preprocess text for analysis."""
        text = str(text)
        text = self._normalize_unicode(text)
        text = self._remove_urls(text)
        text = self._remove_emails(text)
        text = self._remove_html_tags(text)
        text = self._expand_contractions(text)
        text = self._normalize_whitespace(text)
        return text.strip()

    def _normalize_unicode(self, text: str) -> str:
        """Normalize unicode characters."""
        return unicodedata.normalize('NFKD', text)

    def _remove_urls(self, text: str) -> str:
        """Remove URLs from text."""
        url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        return url_pattern.sub('', text)

    def _remove_emails(self, text: str) -> str:
        """Remove email addresses from text."""
        email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
        return email_pattern.sub('', text)

    def _remove_html_tags(self, text: str) -> str:
        """Remove HTML tags from text."""
        html_pattern = re.compile(r'<[^>]+>')
        return html_pattern.sub('', text)

    def _expand_contractions(self, text: str) -> str:
        """Expand contractions in text."""
        for contraction, expansion in self.contractions.items():
            text = text.replace(contraction, expansion)
        return text

    def _normalize_whitespace(self, text: str) -> str:
        """Normalize whitespace in text."""
        return re.sub(r'\s+', ' ', text)

class WatermarkDetector:
    def __init__(self):
        self.patterns = {
            'repetition': self._check_repetition,
            'perplexity': self._check_perplexity,
            'token_patterns': self._check_token_patterns,
            'sentence_structure': self._check_sentence_structure
        }
        self.ai_markers = [
            "as an ai language model", "i am an ai", "i cannot", "i don't have",
            "i don't know", "i'm not sure", "i'm unable to", "i apologize",
            "i'm sorry", "as an artificial intelligence", "i'm an ai assistant"
        ]

    def detect(self, text: str) -> float:
        """Detect watermarks in text."""
        try:
            watermark_score = 0.0
            text_lower = text.lower()

            # Check for AI markers
            marker_count = 0
            for marker in self.ai_markers:
                if marker in text_lower:
                    marker_count += 1

            if marker_count > 0:
                watermark_score += min(marker_count * 0.2, 0.8)

            # Check patterns
            for pattern_name, pattern_func in self.patterns.items():
                try:
                    pattern_score = pattern_func(text)
                    watermark_score += pattern_score * 0.1
                except Exception as e:
                    logger.warning(f"Error in pattern {pattern_name}: {e}")

            return min(watermark_score, 1.0)

        except Exception as e:
            logger.warning(f"Error in watermark detection: {e}")
            return 0.0

    def _check_repetition(self, text: str) -> float:
        """Check for repetitive patterns."""
        words = text.lower().split()
        if len(words) < 10:
            return 0.0

        # Check for repeated phrases
        bigrams = [f"{words[i]} {words[i+1]}" for i in range(len(words)-1)]
        trigrams = [f"{words[i]} {words[i+1]} {words[i+2]}" for i in range(len(words)-2)]

        bigram_counts = Counter(bigrams)
        trigram_counts = Counter(trigrams)

        # High repetition suggests AI
        max_bigram_count = max(bigram_counts.values()) if bigram_counts else 0
        max_trigram_count = max(trigram_counts.values()) if trigram_counts else 0

        repetition_score = 0.0
        if max_bigram_count > 3:
            repetition_score += 0.3
        if max_trigram_count > 2:
            repetition_score += 0.5

        return min(repetition_score, 1.0)

    def _check_perplexity(self, text: str) -> float:
        """Check perplexity indicators."""
        # Simple heuristic: very consistent sentence lengths might indicate AI
        sentences = sent_tokenize(text)
        if len(sentences) < 3:
            return 0.0

        sentence_lengths = [len(word_tokenize(sent)) for sent in sentences]
        if len(sentence_lengths) == 0:
            return 0.0

        # Calculate coefficient of variation
        mean_length = np.mean(sentence_lengths)
        std_length = np.std(sentence_lengths)

        if mean_length == 0:
            return 0.0

        cv = std_length / mean_length

        # Very low variation might indicate AI
        if cv < 0.3:
            return 0.5
        return 0.0

    def _check_token_patterns(self, text: str) -> float:
        """Check for token patterns."""
        # Check for overly formal language patterns
        formal_patterns = [
            r'\b(however|furthermore|moreover|therefore|consequently|nevertheless)\b',
            r'\b(in conclusion|in summary|to summarize|in other words)\b',
            r'\b(it is important to note|it should be noted|it is worth mentioning)\b'
        ]

        pattern_count = 0
        for pattern in formal_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            pattern_count += len(matches)

        # Normalize by text length
        words = len(text.split())
        if words == 0:
            return 0.0

        pattern_density = pattern_count / words
        return min(pattern_density * 10, 1.0)  # Scale up the score

    def _check_sentence_structure(self, text: str) -> float:
        """Check sentence structure patterns."""
        sentences = sent_tokenize(text)
        if len(sentences) < 3:
            return 0.0

        # Check for overly similar sentence starts
        sentence_starts = []
        for sent in sentences:
            words = word_tokenize(sent.lower())
            if len(words) >= 2:
                sentence_starts.append(f"{words[0]} {words[1]}")

        if len(sentence_starts) == 0:
            return 0.0

        start_counts = Counter(sentence_starts)
        max_count = max(start_counts.values())

        # If many sentences start similarly, might be AI
        if max_count > len(sentences) * 0.3:
            return 0.6

        return 0.0

class StylometricAnalyzer:
    def __init__(self):
        try:
            self.stopwords = set(stopwords.words('english'))
        except:
            logger.warning("NLTK stopwords not available, using basic set")
            self.stopwords = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they', 'me', 'him', 'her', 'us', 'them'}

        self.features = {
            'avg_word_length': self._calculate_avg_word_length,
            'avg_sentence_length': self._calculate_avg_sentence_length,
            'type_token_ratio': self._calculate_type_token_ratio,
            'stopword_ratio': self._calculate_stopword_ratio,
            'punctuation_ratio': self._calculate_punctuation_ratio,
            'sentence_complexity': self._calculate_sentence_complexity
        }

    def analyze(self, text: str) -> Dict[str, float]:
        """Analyze stylometric features of text."""
        features = {}
        for feature_name, feature_func in self.features.items():
            try:
                features[feature_name] = feature_func(text)
            except Exception as e:
                logger.warning(f"Error calculating {feature_name}: {e}")
                features[feature_name] = 0.0
        return features

    def _calculate_avg_word_length(self, text: str) -> float:
        """Calculate average word length."""
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalpha()]
        if not words:
            return 0.0
        return sum(len(word) for word in words) / len(words)

    def _calculate_avg_sentence_length(self, text: str) -> float:
        """Calculate average sentence length in words."""
        sentences = sent_tokenize(text)
        if not sentences:
            return 0.0

        total_words = 0
        for sentence in sentences:
            words = word_tokenize(sentence)
            total_words += len([word for word in words if word.isalpha()])

        return total_words / len(sentences)

    def _calculate_type_token_ratio(self, text: str) -> float:
        """Calculate type-token ratio (lexical diversity)."""
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalpha()]
        if not words:
            return 0.0

        unique_words = len(set(words))
        total_words = len(words)
        return unique_words / total_words

    def _calculate_stopword_ratio(self, text: str) -> float:
        """Calculate ratio of stopwords."""
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalpha()]
        if not words:
            return 0.0

        stopword_count = sum(1 for word in words if word in self.stopwords)
        return stopword_count / len(words)

    def _calculate_punctuation_ratio(self, text: str) -> float:
        """Calculate ratio of punctuation marks."""
        if not text:
            return 0.0

        punct_count = sum(1 for char in text if char in string.punctuation)
        return punct_count / len(text)

    def _calculate_sentence_complexity(self, text: str) -> float:
        """Calculate average sentence complexity (clauses per sentence)."""
        sentences = sent_tokenize(text)
        if not sentences:
            return 0.0

        total_clauses = 0
        clause_indicators = [',', ';', ':', 'and', 'but', 'or', 'because', 'since', 'while', 'although', 'though', 'if', 'when', 'where', 'that', 'which', 'who']

        for sentence in sentences:
            clause_count = 1  # Base sentence
            sentence_lower = sentence.lower()
            for indicator in clause_indicators:
                clause_count += sentence_lower.count(indicator)
            total_clauses += clause_count

        return total_clauses / len(sentences)

class FeatureExtractor:
    def __init__(self):
        self.features = {
            'lexical_diversity': self._extract_lexical_diversity,
            'readability_score': self._extract_readability_score,
            'word_frequency': self._extract_word_frequency,
            'sentence_variety': self._extract_sentence_variety,
            'semantic_coherence': self._extract_semantic_coherence
        }

    def extract(self, text: str) -> Dict[str, float]:
        """Extract features from text."""
        features = {}
        for feature_name, feature_func in self.features.items():
            try:
                features[feature_name] = feature_func(text)
            except Exception as e:
                logger.warning(f"Error extracting {feature_name}: {e}")
                features[feature_name] = 0.0
        return features

    def _extract_lexical_diversity(self, text: str) -> float:
        """Extract lexical diversity metrics."""
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalpha()]
        if len(words) < 10:
            return 0.0

        # Calculate TTR (Type-Token Ratio)
        unique_words = len(set(words))
        total_words = len(words)
        ttr = unique_words / total_words

        # Calculate MSTTR (Mean Segmental TTR) for longer texts
        if total_words > 100:
            segment_size = 50
            segments = [words[i:i+segment_size] for i in range(0, len(words), segment_size)]
            segment_ttrs = []
            for segment in segments:
                if len(segment) >= 10:
                    seg_ttr = len(set(segment)) / len(segment)
                    segment_ttrs.append(seg_ttr)
            msttr = np.mean(segment_ttrs) if segment_ttrs else ttr
            return msttr

        return ttr

    def _extract_readability_score(self, text: str) -> float:
        """Extract readability score."""
        try:
            # Try to use textstat if available
            score = flesch_reading_ease(text)
            # Normalize to 0-1 range (higher = more readable = potentially more AI)
            normalized_score = max(0, min(100, score)) / 100
            return normalized_score
        except:
            # Fallback: simple readability metric
            sentences = sent_tokenize(text)
            words = word_tokenize(text)

            if not sentences or not words:
                return 0.0

            avg_sentence_length = len(words) / len(sentences)
            avg_word_length = sum(len(word) for word in words if word.isalpha()) / len([w for w in words if w.isalpha()])

            # Simple readability formula (inverse of complexity)
            complexity = (avg_sentence_length * 0.1) + (avg_word_length * 0.2)
            readability = max(0, min(1, 1 - (complexity / 10)))
            return readability

    def _extract_word_frequency(self, text: str) -> float:
        """Extract word frequency patterns."""
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalpha()]

        if len(words) < 10:
            return 0.0

        word_counts = Counter(words)

        # Calculate frequency distribution metrics
        frequencies = list(word_counts.values())

        # Zipf's law compliance (natural texts follow Zipf distribution)
        sorted_freqs = sorted(frequencies, reverse=True)

        # Calculate how well it fits Zipf's law
        ranks = range(1, len(sorted_freqs) + 1)

        # Expected frequency according to Zipf's law
        total_words = sum(sorted_freqs)
        expected_freqs = [total_words / (rank * sum(1/r for r in ranks)) for rank in ranks]

        # Calculate deviation from Zipf's law
        if len(expected_freqs) > 0:
            mse = np.mean([(actual - expected) ** 2 for actual, expected in zip(sorted_freqs, expected_freqs)])
            max_possible_mse = np.var(sorted_freqs)
            if max_possible_mse > 0:
                zipf_compliance = 1 - (mse / max_possible_mse)
                return max(0, min(1, zipf_compliance))

        return 0.5  # Default if calculation fails

    def _extract_sentence_variety(self, text: str) -> float:
        """Extract sentence variety metrics."""
        sentences = sent_tokenize(text)
        if len(sentences) < 3:
            return 0.0

        # Analyze sentence length variety
        sentence_lengths = [len(word_tokenize(sent)) for sent in sentences]
        length_variety = np.std(sentence_lengths) / (np.mean(sentence_lengths) + 1e-10)

        return length_variety # Higher variety is good, lower might be AI

    def _extract_semantic_coherence(self, text: str) -> float:
        """
        Extract semantic coherence patterns.
        This is a placeholder for a more advanced NLP technique.
        For a simple heuristic, we can check for common introductory/concluding phrases
        that might make the text feel "too perfect" or formulaic.
        """
        coherence_score = 0.0
        text_lower = text.lower()

        # Overly generic or formulaic phrases
        formulaic_phrases = [
            "in conclusion", "to summarize", "in essence", "ultimately",
            "as previously discussed", "this demonstrates that", "it is clear that",
            "it is important to consider"
        ]

        for phrase in formulaic_phrases:
            if phrase in text_lower:
                coherence_score += 0.1 # Add a small penalty for each found

        # Could add checks for repeating concepts too closely without new information
        # (requires more advanced NLP like topic modeling or embeddings)

        # For now, a very simple heuristic: if a text is very short and contains formulaic phrases,
        # it might be a strong indicator.
        words = word_tokenize(text_lower)
        if len(words) < 50 and coherence_score > 0.3:
            coherence_score += 0.3 # Stronger indicator for short, formulaic texts

        return min(coherence_score, 1.0)


class AITextDetector:
    def __init__(self):
        self.preprocessor = TextPreprocessor()
        self.watermark_detector = WatermarkDetector()
        self.stylometric_analyzer = StylometricAnalyzer()
        self.feature_extractor = FeatureExtractor()

        self.feature_weights = {
            'watermark_detection': 0.3,
            'stylometric_analysis': 0.3,
            'feature_extraction': 0.4
        }
        # Thresholds for converting features to AI indicators (heuristics)
        self.stylometric_thresholds = {
            'avg_word_length': {'min': 4.0, 'max': 6.0, 'ai_deviation_score': 0.2}, # AI often has consistent lengths
            'avg_sentence_length': {'min': 15.0, 'max': 25.0, 'ai_deviation_score': 0.2}, # AI often has consistent lengths
            'type_token_ratio': {'min': 0.6, 'max': 0.8, 'ai_deviation_score': 0.2}, # AI can be less diverse or too perfect
            'stopword_ratio': {'min': 0.4, 'max': 0.55, 'ai_deviation_score': 0.1}, # AI can have consistent stopword usage
            'punctuation_ratio': {'min': 0.02, 'max': 0.05, 'ai_deviation_score': 0.1}, # AI can have consistent punctuation
            'sentence_complexity': {'min': 1.2, 'max': 1.8, 'ai_deviation_score': 0.1} # AI can have consistent complexity
        }
        self.extracted_feature_thresholds = {
            'lexical_diversity': {'min': 0.6, 'max': 0.8, 'ai_deviation_score': 0.3},
            'readability_score': {'min': 0.6, 'max': 0.9, 'ai_deviation_score': 0.3}, # Very high readability might indicate AI
            'word_frequency': {'min': 0.8, 'max': 1.0, 'ai_deviation_score': 0.2}, # Deviation from Zipf's law can indicate AI
            'sentence_variety': {'min': 0.1, 'max': 0.5, 'ai_deviation_score': 0.2}, # Low variety might indicate AI
            'semantic_coherence': {'min': 0.0, 'max': 0.3, 'ai_deviation_score': 0.3} # High score on semantic coherence indicates AI
        }


    def analyze_text(self, text: str) -> Dict:
        """Analyze text to determine if it's AI-generated."""
        preprocessed_text = self.preprocessor.preprocess(text)

        if not preprocessed_text.strip():
            return {
                'is_ai_generated': False,
                'confidence_score': 0.0,
                'ai_indicators': {},
                'detailed_scores': {},
                'error': "Input text is empty or too short after preprocessing."
            }

        try:
            watermark_score = self.watermark_detector.detect(preprocessed_text)
            stylometric_features = self.stylometric_analyzer.analyze(preprocessed_text)
            extracted_features = self.feature_extractor.extract(preprocessed_text)

            ai_indicators = {}
            detailed_scores = {
                'watermark_detection': float(watermark_score),
                'stylometric_analysis': stylometric_features,
                'feature_extraction': extracted_features
            }

            # Convert stylometric features to AI indicators
            stylometric_ai_score = 0.0
            for feature, value in stylometric_features.items():
                if feature in self.stylometric_thresholds:
                    thresholds = self.stylometric_thresholds[feature]
                    if not (thresholds['min'] <= value <= thresholds['max']):
                        stylometric_ai_score += thresholds['ai_deviation_score']
                        ai_indicators[f'stylometric_anomaly_{feature}'] = True
                    else:
                         ai_indicators[f'stylometric_anomaly_{feature}'] = False
            stylometric_ai_score = min(stylometric_ai_score, 1.0)
            detailed_scores['stylometric_combined_score'] = float(stylometric_ai_score)


            # Convert extracted features to AI indicators
            extracted_ai_score = 0.0
            # Lower lexical diversity might be AI
            if extracted_features['lexical_diversity'] < self.extracted_feature_thresholds['lexical_diversity']['min']:
                extracted_ai_score += self.extracted_feature_thresholds['lexical_diversity']['ai_deviation_score']
                ai_indicators['low_lexical_diversity'] = True
            elif extracted_features['lexical_diversity'] > self.extracted_feature_thresholds['lexical_diversity']['max']:
                 # Or unusually high (if AI aims for "perfect" diversity)
                extracted_ai_score += self.extracted_feature_thresholds['lexical_diversity']['ai_deviation_score'] * 0.5 # Less severe penalty
                ai_indicators['unusually_high_lexical_diversity'] = True
            else:
                ai_indicators['low_lexical_diversity'] = False


            # Very high readability might be AI
            if extracted_features['readability_score'] > self.extracted_feature_thresholds['readability_score']['max']:
                extracted_ai_score += self.extracted_feature_thresholds['readability_score']['ai_deviation_score']
                ai_indicators['unusually_high_readability'] = True
            else:
                ai_indicators['unusually_high_readability'] = False

            # Deviation from Zipf's law (word_frequency) might indicate AI
            if extracted_features['word_frequency'] < self.extracted_feature_thresholds['word_frequency']['min']:
                extracted_ai_score += self.extracted_feature_thresholds['word_frequency']['ai_deviation_score']
                ai_indicators['zipf_deviation'] = True
            else:
                ai_indicators['zipf_deviation'] = False

            # Low sentence variety might indicate AI
            if extracted_features['sentence_variety'] < self.extracted_feature_thresholds['sentence_variety']['min']:
                extracted_ai_score += self.extracted_feature_thresholds['sentence_variety']['ai_deviation_score']
                ai_indicators['low_sentence_variety'] = True
            else:
                ai_indicators['low_sentence_variety'] = False

            # High semantic coherence score indicates AI
            if extracted_features['semantic_coherence'] > self.extracted_feature_thresholds['semantic_coherence']['max']:
                extracted_ai_score += self.extracted_feature_thresholds['semantic_coherence']['ai_deviation_score']
                ai_indicators['high_semantic_coherence'] = True
            else:
                ai_indicators['high_semantic_coherence'] = False


            extracted_ai_score = min(extracted_ai_score, 1.0)
            detailed_scores['extracted_features_combined_score'] = float(extracted_ai_score)


            # Calculate final score
            final_score = (
                self.feature_weights['watermark_detection'] * watermark_score +
                self.feature_weights['stylometric_analysis'] * stylometric_ai_score +
                self.feature_weights['feature_extraction'] * extracted_ai_score
            )
            final_score = min(final_score, 1.0)

            # Add AI indicator bonus
            ai_indicator_bonus = sum(ai_indicators.values()) * 0.1 # Small bonus for each positive indicator
            final_score = min(final_score + ai_indicator_bonus, 1.0)


            return {
                'is_ai_generated': bool(final_score > 0.55),
                'confidence_score': float(final_score),
                'ai_indicators': ai_indicators,
                'detailed_scores': detailed_scores,
                'visualizations': self._generate_text_visualizations(
                    watermark_score, stylometric_ai_score, extracted_ai_score, final_score
                )
            }

        except Exception as e:
            logger.error(f"Error analyzing text: {str(e)}")
            return {
                'is_ai_generated': False,
                'confidence_score': 0.0,
                'error': str(e)
            }

    def _generate_text_visualizations(self, watermark_score, stylometric_score, extracted_features_score, final_score):
        """Generate visualizations for text analysis results."""
        visualizations = {}

        try:
            # Scores bar chart
            scores = {
                'Watermark Detection': watermark_score,
                'Stylometric Analysis': stylometric_score,
                'Feature Extraction': extracted_features_score
            }

            fig_scores = px.bar(
                x=list(scores.keys()),
                y=list(scores.values()),
                labels={'x': 'Analysis Type', 'y': 'Score'},
                title='Text Analysis Scores',
                color=list(scores.values()),
                color_continuous_scale='RdYlBu_r'
            )
            fig_scores.update_layout(showlegend=False)
            visualizations['analysis_scores'] = fig_scores

            # Confidence gauge
            fig_gauge = go.Figure(go.Indicator(
                mode="gauge+number",
                value=final_score,
                domain={'x': [0, 1], 'y': [0, 1]},
                title={'text': "AI Generation Confidence"},
                gauge={'axis': {'range': [0, 1]},
                       'steps': [
                           {'range': [0, 0.3], 'color': "lightgreen"},
                           {'range': [0.3, 0.7], 'color': "yellow"},
                           {'range': [0.7, 1], 'color': "lightcoral"}],
                       'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 0.55}}
            ))
            visualizations['confidence_gauge'] = fig_gauge

        except Exception as e:
            logger.warning(f"Error generating text visualizations: {e}")

        return visualizations


# Initialize detectors globally or within the Gradio function to avoid re-initialization
image_detector = ImageDetector()
text_detector = AITextDetector()

def ai_content_detector_interface(image: Image.Image, text_input: str):
    """
    Main function for the Gradio interface that handles both image and text analysis.
    """
    image_result = None
    text_result = None

    # Handle image analysis
    if image is not None:
        # Gradio passes PIL.Image directly, save it temporarily
        image_path = "temp_image.png"
        image.save(image_path)
        image_result = image_detector.analyze_image(image_path)
        os.remove(image_path) # Clean up temp file
    else:
        image_result = {
            'is_ai_generated': False,
            'confidence_score': 0.0,
            'error': "No image provided for analysis."
        }

    # Handle text analysis
    if text_input and text_input.strip():
        text_result = text_detector.analyze_text(text_input)
    else:
        text_result = {
            'is_ai_generated': False,
            'confidence_score': 0.0,
            'error': "No text provided for analysis."
        }

    # Format output for Gradio
    image_output_str = f"**Image Analysis:**\n" \
                       f"Is AI Generated: {image_result['is_ai_generated']}\n" \
                       f"Confidence Score: {image_result['confidence_score']:.2f}\n"

    if 'error' in image_result:
        image_output_str += f"Error: {image_result['error']}\n"
    else:
        image_output_str += f"AI Indicators: {json.dumps(image_result['ai_indicators'], indent=2, cls=NumpyEncoder)}\n" \
                            f"Detailed Scores: {json.dumps(image_result['detailed_scores'], indent=2, cls=NumpyEncoder)}\n"

    text_output_str = f"**Text Analysis:**\n" \
                      f"Is AI Generated: {text_result['is_ai_generated']}\n" \
                      f"Confidence Score: {text_result['confidence_score']:.2f}\n"

    if 'error' in text_result:
        text_output_str += f"Error: {text_result['error']}\n"
    else:
        text_output_str += f"AI Indicators: {json.dumps(text_result['ai_indicators'], indent=2, cls=NumpyEncoder)}\n" \
                           f"Detailed Scores: {json.dumps(text_result['detailed_scores'], indent=2, cls=NumpyEncoder)}\n"


    # Prepare visualizations for Gradio's Plot component
    image_analysis_scores_plot = image_result.get('visualizations', {}).get('analysis_scores')
    image_confidence_gauge_plot = image_result.get('visualizations', {}).get('confidence_gauge')

    text_analysis_scores_plot = text_result.get('visualizations', {}).get('analysis_scores')
    text_confidence_gauge_plot = text_result.get('visualizations', {}).get('confidence_gauge')

    return (image_output_str, image_analysis_scores_plot, image_confidence_gauge_plot,
            text_output_str, text_analysis_scores_plot, text_confidence_gauge_plot)


# Create the Gradio interface
iface = gr.Interface(
    fn=ai_content_detector_interface,
    inputs=[
        gr.Image(type="pil", label="Upload Image (Optional)"),
        gr.Textbox(lines=10, label="Enter Text Here (Optional)")
    ],
    outputs=[
        gr.Textbox(label="Image Analysis Result"),
        gr.Plot(label="Image Analysis Scores"),
        gr.Plot(label="Image Confidence Gauge"),
        gr.Textbox(label="Text Analysis Result"),
        gr.Plot(label="Text Analysis Scores"),
        gr.Plot(label="Text Confidence Gauge")
    ],
    title="AI Generated Content Detector",
    description="Upload an image and/or enter text to detect if it's AI-generated.",
    live=False # Set to True if you want live updates as text is typed
)

# Launch the Gradio app
iface.launch(share=True) # `share=True` generates a public link

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 306MB/s]


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c183f173347c49e74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
